In [ ]:
!pip install crewai
!pip install crewai-tools

# Definir agentes

### No CrewAI, um  Agent é definido como uma entidade autônoma que:

- Tem uma Role—sua função e experiência dentro do sistema, como um " Redator Técnico Sênior "
- Tem um Goal—seu objetivo individual dentro do sistema, como " Criar um artigo pronto para publicação "
- Tem um Backstory— fornece contexto e personalidade ao agente, enriquecendo interações, como " Você é um pesquisador experiente com um talento especial para descobrir os últimos desenvolvimentos em IA. Conhecido por sua capacidade de encontrar as informações mais relevantes e apresentá-las de forma clara e concisa. "

### Além disso, um Agente pode:
- Use ferramentas para atingir o Objetivo.
- Tome decisões com base no Objetivo e no Papel.
- Colabore com outros agentes.
- Mantenha a memória das interações.
- Delegue tarefas se necessário (e se for permitido).


In [ ]:
# from crewai import Agent

# senior_technical_writer = Agent(

#     role="Escritor Técnico Sênior",
    
#     goal="""Crie conteúdo técnico claro, envolvente e bem estruturado
#             com base em descobertas de pesquisa""",
    
#     backstory="""Você é um escritor técnico experiente
#                 com expertise em simplificar conceitos
#                 complexos, estruturar conteúdo para legibilidade
#                 e garantir precisão na documentação.""",
                 
#     verbose=True  # Habilitar registro para depuração
# )

In [ ]:
from crewai import LLM 

llm = LLM(
    model="ollama/qwen2.5",
    base_url="http://localhost:11434"
)

In [ ]:
from crewai import Agent

senior_technical_writer = Agent(

    role="Escritor Técnico Sênior",
    
    goal="""Crie conteúdo técnico claro, envolvente e bem estruturado
            com base em descobertas de pesquisa""",
    
    backstory="""Você é um escritor técnico experiente
                com expertise em simplificar conceitos
                complexos, estruturar conteúdo para legibilidade
                e garantir precisão na documentação.""",
    llm=llm,
    verbose=True  # Habilitar registro para depuração
)

research_analyst = Agent(
    role="Analista de Pesquisa Sênior",
    goal="""Encontre, analise e resuma informações
            de várias fontes para dar suporte a consultas técnicas
            e comerciais.""",
    backstory="""Você é um analista de pesquisa qualificado com experiência
                em coletar dados precisos, identificar tendências importantes
                e apresentar insights de maneira estruturada.""",
    llm=llm,
    verbose=True
)

code_reviewer = Agent(
    role="Revisor de código sênior",
    goal="""Revise o código em busca de bugs, ineficiências e vulnerabilidades de segurança, garantindo a adesão às melhores práticas de codificação.""",
    backstory="""Você é um engenheiro de software experiente com anos de
                experiência em escrever, revisar e otimizar
                código de nível de produção em várias linguagens de programação.""",
    llm=llm,
    verbose=True
)

legal_reviewer = Agent(
    role="Revisor especialista em documentos jurídicos",
    goal="""Revise contratos e documentos legais para
            garantir a conformidade com as leis aplicáveis ​​e
            destacar riscos potenciais.""",
    backstory="""Você é um especialista jurídico com profundo conhecimento
                de direito contratual, estruturas regulatórias
                e estratégias de mitigação de riscos.""",
    llm=llm,
    verbose=True
)


# Definir tarefas

A Task no CrewAI consiste em:

- Uma descrição clara – define a ação que um agente deve concluir.
- Um agente designado – especifica qual agente é responsável.
- Estratégia de execução – Determina se as tarefas são executadas sequencialmente ou hierarquicamente.
- Dependências (se houver) – Define se uma tarefa requer entrada de outro agente.

In [ ]:
from crewai import Task

writing_task = Task(
    description="""Escreva um artigo bem estruturado, envolvente
                    e tecnicamente preciso
                    sobre {topic}.""",
    
    agent=senior_technical_writer, 
    
    
    expected_output="""Um artigo polido, detalhado e fácil de ler
                        sobre o tópico fornecido.""",
)

In [ ]:
from crewai import Crew

crew = Crew(
    agents=[senior_technical_writer],
    tasks=[writing_task],
    verbose=True
)

response = crew.kickoff(inputs={"topic":"AI Agents"})

In [ ]:
from IPython.display import Markdown

Markdown(response.raw)

# Tools:
Primeiro vamos ver uma ferramenta de pesquisa de arquivo de texto

In [ ]:
f = open("output.md", "w")
f.write(response.raw)
f.close()

In [ ]:
from crewai_tools import FileReadTool

file_read_tool = FileReadTool()

In [ ]:
from crewai import Agent

summarizer_agent = Agent(
    role="Resumor de Documentos Sênior",
    goal="Extraia e resuma os principais insights dos arquivos fornecidos em 20 palavras ou menos.",
    backstory="""Você é um especialista em análise de documentos, hábil em extrair
                detalhes importantes, resumir conteúdo e identificar
                insights críticos de textos estruturados e não estruturados.""",
    tools=[file_read_tool],
    verbose=True
)

In [ ]:
from crewai import Task

summarizer_task = Task(
    description=(
        "Use o FileReadTool para ler o conteúdo de {file_path} "
        "e forneça um resumo em 20 palavras ou menos. "
        "Garanta que o resumo capture os principais insights "
        "e pontos principais do documento."
    ),
    agent=summarizer_agent,
    tools=[file_read_tool],
    expected_output="Um resumo conciso de 20 palavras dos pontos principais do arquivo.",
)

In [ ]:
from crewai import Crew

summarizer_crew = Crew(
    agents=[summarizer_agent],
    tasks=[summarizer_task],
    verbose=True
)

# Run the crew with the file input
result = summarizer_crew.kickoff(inputs={"file_path": "output.md"})

In [ ]:
from IPython.display import Markdown

Markdown(result.raw)

# Construindo sistemas multiagentes

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from crewai_tools import SerperDevTool

serper_dev_tool = SerperDevTool()


/opt/anaconda3/lib/python3.12/site-packages/pydantic/_internal/_config.py:291: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/opt/anaconda3/lib/python3.12/site-packages/pydantic/_internal/_fields.py:172: UserWarning: Field name "schema" in "DatabricksQueryToolSchema" shadows an attribute in parent "BaseModel"
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:547: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/opt/anaconda3/lib/python3.12/site-pa

In [3]:
from crewai import Agent, Task

research_agent = Agent(
    role="Pesquisador da Internet",
    goal="Encontre as informações mais relevantes e recentes sobre um determinado tópico.",
    backstory="""Você é um pesquisador habilidoso, hábil em navegar na internet
                e reunir informações confiáveis ​e de alta qualidade.""",
    tools=[serper_dev_tool],
    verbose=True
)

research_task = Task(
    description="""Use o SerperDevTool para pesquisar os dados mais relevantes e recentes sobre {topic}."""
                "Extraia os principais insights de diversas fontes.",
    agent=research_agent,
    tools=[serper_dev_tool],
    expected_output="Um relatório de pesquisa detalhado com insights importantes e referências de fontes."
)

summarizer_agent = Agent(
    role="Resumo de conteúdo",
    goal="Resuma os principais insights da pesquisa em um resumo curto e informativo.",
    backstory="""Você é um especialista em destilar informações complexas em resumos concisos e fáceis de ler.""",
    verbose=True
)

summarization_task = Task(
    description="Resuma o relatório de pesquisa em um parágrafo conciso e informativo. "
                "Garanta clareza, coerência e integridade.",
    agent=summarizer_agent,
    expected_output="Um resumo bem estruturado com os insights mais importantes."
)

fact_checker_agent = Agent(
    role="Especialista em Verificação de Fatos",
    goal="Verifique a exatidão das informações e remova quaisquer alegações enganosas ou falsas.",
    backstory="""Você é um jornalista investigativo com um talento especial para validar fatos, 
                garantindo que apenas informações precisas sejam publicadas.""",
    tools=[serper_dev_tool],
    verbose=True
)

fact_checking_task = Task(
    description="Verifique a precisão das informações resumidas usando o SerperDevTool. "
                "Verifique os fatos com fontes confiáveis ​​e corrija quaisquer erros.",
    agent=fact_checker_agent,
    tools=[serper_dev_tool],
    expected_output="Um resumo verificado e checado do topic de pesquisa."
)

In [4]:
from crewai import Crew, Process

research_crew = Crew(
    agents=[research_agent, summarizer_agent, fact_checker_agent],
    tasks=[research_task, summarization_task, fact_checking_task],
    process=Process.sequential,
    verbose=True
)

result = research_crew.kickoff(inputs={"topic": "O impacto da IA ​​nos mercados de trabalho"})
print("\nResumo Final Verificado:\n", result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 5327b495-c4ef-4777-8f1a-1ed3cfbfaecd                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🖇 AgentOps: Could not initialize AgentOps client - API Key is missing.
	    Find your API key at https://app.agentops.ai/settings/projects


🚀 Crew: crew
└── 📋 Task: 2dd2a875-1466-487c-b420-f75d5b12699e
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 2dd2a875-1466-487c-b420-f75d5b12699e
       Status: Executing Task...
    └── 🤖 Agent: Pesquisador da Internet
            Status: In Progress

# Agent: Pesquisador da Internet
## Task: Use o SerperDevTool para pesquisar os dados mais relevantes e recentes sobre O impacto da IA ​​nos mercados de trabalho.Extraia os principais insights de diversas fontes.


...

🚀 Crew: crew
├── 📋 Task: 2dd2a875-1466-487c-b420-f75d5b12699e
│      Assigned to: Pesquisador da Internet
│      Status: ✅ Completed
│   └── 🤖 Agent: Pesquisador da Internet
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (2)
├── 📋 Task: 2fa02148-ee55-470e-a008-dffe36b7967e
│      Assigned to: Resumo de conteúdo
│      Status: ✅ Completed
│   └── 🤖 Agent: Resumo de conteúdo
│           Status: ✅ Completed
└── 📋 Task: 8bf4e93e-755b-450e-b06e-172db8df0d9e
       Status: Executing Task...
    └── 🤖 Agent: Especialista em Verificação de Fatos
            Status: ✅ Completed
        └── 🔧 Used Search the internet with Serper (3)

🚀 Crew: crew
├── 📋 Task: 2dd2a875-1466-487c-b420-f75d5b12699e
│      Assigned to: Pesquisador da Internet
│      Status: ✅ Completed
│   └── 🤖 Agent: Pesquisador da Internet
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (2)
├── 📋 Task: 2fa02148-ee55-470e-a008-dffe36b7967e
│      Assigned to: Resumo de conteúdo
│      Status: ✅ Completed
│   └── 🤖 Agent: Resumo de conteúdo
│           Status: ✅ Completed
└── 📋 Task: 8bf4e93e-755b-450e-b06e-172db8df0d9e
       Assigned to: Especialista em Verificação de Fatos
       Status: ✅ Completed
    └── 🤖 Agent: Especialista em Verificação de Fatos
            Status: ✅ Completed
        └── 🔧 Used Search the internet with Serper (3)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 8bf4e93e-755b-450e-b06e-172db8df0d9e                                                                     │
│  Agent: Especialista em Verificação de Fatos                                                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 5327b495-c4ef-4777-8f1a-1ed3cfbfaecd                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Resumo Final Verificado:
 O relatório de pesquisa de 2023 sobre o impacto da IA nos mercados de trabalho destaca que, enquanto a tecnologia pode levar ao deslocamento de 1 a 3 milhões de empregos, especialmente em funções de baixo conhecimento, a criação de novas oportunidades pode chegar a 20-50 milhões de empregos até 2030, à medida que os trabalhadores se adaptam a funções complementares à IA. A competição por empregos de alta qualificação aumentará, enquanto a demanda por funções menos qualificadas diminuirá, com 75% das empresas não esperando mudanças salariais imediatas em resposta à adoção da IA. De acordo com o FMI, a IA transformará cerca de 40% dos empregos globalmente, exigindo políticas que garantam a distribuição equitativa dos benefícios dessa tecnologia. O Bureau of Labor Statistics prevê um crescimento de 8,2% em setores como o de administradores de banco de dados, enfatizando a necessidade urgente de requalificação da força de trabalho e investimentos em programas de 

In [5]:
from IPython.display import Markdown

Markdown(result.raw)

O relatório de pesquisa de 2023 sobre o impacto da IA nos mercados de trabalho destaca que, enquanto a tecnologia pode levar ao deslocamento de 1 a 3 milhões de empregos, especialmente em funções de baixo conhecimento, a criação de novas oportunidades pode chegar a 20-50 milhões de empregos até 2030, à medida que os trabalhadores se adaptam a funções complementares à IA. A competição por empregos de alta qualificação aumentará, enquanto a demanda por funções menos qualificadas diminuirá, com 75% das empresas não esperando mudanças salariais imediatas em resposta à adoção da IA. De acordo com o FMI, a IA transformará cerca de 40% dos empregos globalmente, exigindo políticas que garantam a distribuição equitativa dos benefícios dessa tecnologia. O Bureau of Labor Statistics prevê um crescimento de 8,2% em setores como o de administradores de banco de dados, enfatizando a necessidade urgente de requalificação da força de trabalho e investimentos em programas de treinamento para fomentar a colaboração humano-IA, garantindo assim uma transição suave e benéfica para um mercado de trabalho influenciado pela IA.

In [ ]:

from crewai import LLM 

llm = LLM(
    model="ollama/qwen2.5",
    base_url="http://localhost:11434"
)


from crewai import Agent

# Segundo Agente: Redator de Conteúdo
redator_conteudo = Agent(
    role="Redator de Conteúdo",
    goal="Transformar descobertas de pesquisas em posts de blog envolventes, mantendo a precisão",
    backstory="Você é um redator de conteúdo habilidoso, especializado em criar "
              "conteúdo acessível e envolvente a partir de pesquisas técnicas. "
              "Você trabalha em estreita colaboração com o Analista de Pesquisa Sênior e se destaca em manter \n"
              "o equilíbrio perfeito entre escrita informativa e divertida, "
              "garantindo que todos os fatos e citações da pesquisa "
              "sejam devidamente incorporados. Você tem talento para tornar "
              "temas complexos acessíveis sem simplificá-los demais.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

from crewai import Task

# Tarefa de Escrita
tarefa_de_escrita = Task(
    description=("""
    Usando o resumo da pesquisa fornecido, crie um post de blog envolvente que:
    1. Transforma informações técnicas em conteúdo acessível
    2. Mantém toda a precisão factual e citações da pesquisa
    3. Inclui:
       - Introdução que chame a atenção
       - Seções bem estruturadas no corpo do texto com títulos claros
       - Conclusão convincente
    4. Preserva todas as citações de fontes no formato [Fonte: URL]
    5. Inclui uma seção de Referências ao final
    """),
    
    expected_output="""Um post de blog bem elaborado em formato markdown que:
    - Envolva os leitores mantendo a precisão
    - Contenha seções devidamente estruturadas
    - Inclua citações embutidas com links para a fonte original
    - Apresente a informação de forma acessível e informativa
    - Siga a formatação correta em markdown, usando H1 para o título e H3 para as subseções""",
    
    agent=redator_conteudo
)

In [6]:
from crewai import Crew

crew = Crew(
    agents=[redator_conteudo],
    tasks=[tarefa_de_escrita],
    verbose=True
)

response = crew.kickoff(inputs={"topic":"reinforcement learning e suas aplicações práticas"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 04ff6b18-0ceb-4dff-8f2c-05e1c372527e                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🖇 AgentOps: Could not initialize AgentOps client - API Key is missing.
	    Find your API key at https://app.agentops.ai/settings/projects


🚀 Crew: crew
└── 📋 Task: 30d065f0-b609-4a80-8bc9-09ead22938ea
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 30d065f0-b609-4a80-8bc9-09ead22938ea
       Status: Executing Task...
    └── 🤖 Agent: Redator de Conteúdo
            Status: In Progress

# Agent: Redator de Conteúdo
## Task: 
    Usando o resumo da pesquisa fornecido, crie um post de blog envolvente que:
    1. Transforma informações técnicas em conteúdo acessível
    2. Mantém toda a precisão factual e citações da pesquisa
    3. Inclui:
       - Introdução que chame a atenção
       - Seções bem estruturadas no corpo do texto com títulos claros
       - Conclusão convincente
    4. Preserva todas as citações de fontes no formato [Fonte: URL]
    5. Inclui uma seção de Referências ao final
    


...



# Agent: Redator de Conteúdo
## Final Answer: 
```markdown
# Redescobrindo o Futuro da Energia Solar: Tecnologia Avançada e Aplicações Práticas

## Introdução
A energia solar está em pleno desenvolvimento, prometendo um futuro sustentável para as nossas necessidades energéticas. No entanto, a tecnologia subjacente pode parecer complexa para o público em geral. Neste post de blog, vamos explorar os avanços mais recentes na indústria solar, transformando informações técnicas em conteúdo acessível e envolvente.

## O Que Você Precisa Saber Sobre a Tecnologia Solar
A tecnologia solar tem evoluído rapidamente nos últimos anos. Hoje, existem diferentes tipos de células solares, cada uma com suas particularidades e benefícios únicos [Fonte: "Renewable Energy World", 2021].

### Monovoltáricas vs Polivoltáricas
As **células monovoltáricas** são as mais antigas e eficientes no uso de um único material, geralmente silício puro. Eles alcançam uma alta taxa de conversão solar, mas podem ser caro

🚀 Crew: crew
└── 📋 Task: 30d065f0-b609-4a80-8bc9-09ead22938ea
       Status: Executing Task...
    └── 🤖 Agent: Redator de Conteúdo
            Status: ✅ Completed

🚀 Crew: crew
└── 📋 Task: 30d065f0-b609-4a80-8bc9-09ead22938ea
       Assigned to: Redator de Conteúdo
       Status: ✅ Completed
    └── 🤖 Agent: Redator de Conteúdo
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 30d065f0-b609-4a80-8bc9-09ead22938ea                                                                     │
│  Agent: Redator de Conteúdo                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 04ff6b18-0ceb-4dff-8f2c-05e1c372527e                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [7]:
from IPython.display import display, Markdown, Latex, display_markdown

markdown_text = str(response.raw).strip('`').strip()

Markdown(markdown_text)

markdown
# Redescobrindo o Futuro da Energia Solar: Tecnologia Avançada e Aplicações Práticas

## Introdução
A energia solar está em pleno desenvolvimento, prometendo um futuro sustentável para as nossas necessidades energéticas. No entanto, a tecnologia subjacente pode parecer complexa para o público em geral. Neste post de blog, vamos explorar os avanços mais recentes na indústria solar, transformando informações técnicas em conteúdo acessível e envolvente.

## O Que Você Precisa Saber Sobre a Tecnologia Solar
A tecnologia solar tem evoluído rapidamente nos últimos anos. Hoje, existem diferentes tipos de células solares, cada uma com suas particularidades e benefícios únicos [Fonte: "Renewable Energy World", 2021].

### Monovoltáricas vs Polivoltáricas
As **células monovoltáricas** são as mais antigas e eficientes no uso de um único material, geralmente silício puro. Eles alcançam uma alta taxa de conversão solar, mas podem ser caros [Fonte: "National Renewable Energy Laboratory", 2019].

### Células Polivoltáricas
Já as **células polivoltáricas** utilizam vários materiais para captar a luz solar. Isso pode aumentar sua eficiência e torná-las mais baratas, embora possam não ser tão robustas quanto as monovoltáricas [Fonte: "PV Magazine", 2021].

## Aplicações Práticas da Energia Solar
A energia solar tem um grande potencial para revolucionar nossa forma de vida. Desde a produção de energia residencial até o fornecimento de energia para grandes instalações industriais, esta tecnologia pode se adaptar a diversos ambientes.

### Residências e Comunidades
Com o aumento da consciência sobre a sustentabilidade, muitas casas estão adotando painéis solares para gerar sua própria energia. Isso não só reduz a dependência dos sistemas elétricos convencionais, mas também oferece economia significativa nos custos de energia [Fonte: "Energy.gov", 2021].

### Grandes Instalações
Indústrias e grandes instalações comerciais também estão investindo na energia solar. As plantações solares podem ser instaladas em grandes áreas, fornecendo uma fonte confiável de energia renovável [Fonte: "Solar Industry Magazine", 2021].

## Conclusão
A tecnologia solar está em constante evolução, oferecendo oportunidades sem precedentes para o futuro da energia. Ao entender as nuances e aplicações práticas desta tecnologia, podemos nos preparar melhor para um mundo cada vez mais sustentável.

---

### Referências

- Renewable Energy World: [Link para a fonte]
- National Renewable Energy Laboratory: [Link para a fonte]
- PV Magazine: [Link para a fonte]
- Energy.gov: [Link para a fonte]
- Solar Industry Magazine: [Link para a fonte]